#Stock Indicators Python

In [1]:
import pandas as pd
import requests
import numpy as np
from lightweight_charts import Chart
from stock_indicators import indicators, Quote
from datetime import datetime, timedelta
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [2]:
# import yfinance as yf
# df = yf.download('SPY', start='2010-01-01', multi_level_index=False)
# df.reset_index(inplace=True)
# df.to_csv('SPY.csv', index=False)
df = pd.read_csv('SPY.csv')
rawdf = df.copy()
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Close,High,Low,Open,Volume
0,2010-01-04,85.279243,85.324391,83.909719,84.556858,118944600
1,2010-01-05,85.504967,85.542593,84.918029,85.226551,111579900
2,2010-01-06,85.565155,85.775850,85.354460,85.422181,116074400
3,2010-01-07,85.926323,86.031670,85.166311,85.407106,131091100
4,2010-01-08,86.212280,86.249907,85.527521,85.700590,126402800


In [3]:
quotes = [
    Quote(d, o, h, l, c, v)
    for d, o, h, l, c, v in zip(
        df['Date'],
        df['Open'],
        df['High'],
        df['Low'],
        df['Close'],
        df['Volume']
    )
]


In [ ]:
# ATR Trailing Stop
df['atr_stop'] = [r.atr_stop for r in indicators.get_atr_stop(quotes)]
df['atr_stop'] = df['atr_stop'].astype(float) 
df.to_json()
df['bullishATRStop'] = 0.0
df['bullishATRStop'] = np.where(df['Close'] > df['atr_stop'], 1.0, 0.0)
df['crossover_ATRStop'] = df['bullishATRStop'].diff()

# Average Directional Index (ADX)
df['adx'] = [r.adx for r in indicators.get_adx(quotes, 14)]
df['adx_pdi'] = [r.pdi for r in indicators.get_adx(quotes, 14)]
df['adx_mdi'] = [r.mdi for r in indicators.get_adx(quotes, 14)]
df['bullishADX'] = 0.0
df['bullishADX'] = np.where(df['adx_pdi'] > df['adx_mdi'], 1.0, 0.0)
df['crossover_ADX'] = df['bullishADX'].diff()   


# Moving Average Convergence / Divergence (MACD)
df['macd'] = [r.macd for r in indicators.get_macd(quotes, 12, 26, 9)]
df['signal'] = [r.signal for r in indicators.get_macd(quotes, 12, 26, 9)]
df['histogram'] = [r.histogram for r in indicators.get_macd(quotes, 12, 26, 9)]
df['bullishMACD'] = 0.0
df['bullishMACD'] = np.where(df['macd'] > df['signal'], 1.0, 0.0)
df['crossover_MACD'] = df['bullishMACD'].diff()

# Williams Alligator
df['jaw'] = [r.jaw for r in indicators.get_alligator(quotes, 13, 8, 8, 5, 5, 3)]
df['teeth'] = [r.teeth for r in indicators.get_alligator(quotes, 13, 8, 8, 5, 5, 3)]
df['lips'] = [r.lips for r in indicators.get_alligator(quotes, 13, 8, 8, 5, 5, 3)]

# Vortex Indicator (VI)
df['pvi'] = [r.pvi for r in indicators.get_vortex(quotes, 14)]
df['mvi'] = [r.mvi for r in indicators.get_vortex(quotes, 14)]
df['bullishVI'] = 0.0
df['bullishVI'] = np.where(df['pvi'] > df['mvi'], 1.0, 0.0)
df['crossover_VI'] = df['bullishVI'].diff()   




# Calculate EMA
df['EMA 12'] = [r.ema for r in indicators.get_ema(quotes, 12)]
df['EMA 20'] = [r.ema for r in indicators.get_ema(quotes, 20)]
df['EMA 25'] = [r.ema for r in indicators.get_ema(quotes, 25)]

# Calculate EMA12 crossover EMA25
df['bullishEMA'] = 0.0
df['bullishEMA'] = np.where(df['EMA 12'] > df['EMA 25'], 1.0, 0.0)
df['crossover_EMA12_EMA25'] = df['bullishEMA'].diff()

# Stochastic Momentum Index (SMI)
df['smi'] = [r.smi for r in indicators.get_smi(quotes, 14, 3, 3)]
df['smi_signal'] = [r.signal for r in indicators.get_smi(quotes, 14, 3, 3)]
df['bullishSMI'] = 0.0
df['bullishSMI'] = np.where(df['smi'] > df['smi_signal'], 1.0, 0.0)
df['crossover_SMI'] = df['bullishSMI'].diff()   

# Calculate Chandelier exit
from stock_indicators import ChandelierType 
df['chandelier_long_exit'] = [r.chandelier_exit for r in indicators.get_chandelier(quotes, 22, 3, ChandelierType.LONG)]
df['chandelier_short_exit'] = [r.chandelier_exit for r in indicators.get_chandelier(quotes, 22, 3, ChandelierType.SHORT)]
df['chandelier_exit'] = 0
df['chandelier_exit'] = np.where(df['chandelier_long_exit'] > df['chandelier_short_exit'], 1.0, 0.0)
df['crossover_chandelier_exit'] = df['chandelier_exit'].diff()  

# Calculate Bollinger Bands
df['BB_upper_band'] = [r.upper_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['BB_middle_band'] = [r.sma for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['BB_lower_band'] = [r.lower_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]

# Arnaud Legoux Moving Average (ALMA)
df['ALMA_21'] = [r.alma for r in indicators.get_alma(quotes, 21, 0.85, 6)]
df['ALMA_50'] = [r.alma for r in indicators.get_alma(quotes, 50, 0.85, 6)]
df['alma_bullish'] = 0.0
df['alma_bullish'] = np.where(df['ALMA_21'] > df['ALMA_50'], 1.0, 0.0)
df['crossover_ALMA'] = df['alma_bullish'].diff()

# Ehlers Fisher Transform
df['fisher'] = [r.fisher for r in indicators.get_fisher_transform(quotes, 10)]
df['fisher_trigger'] = [r.trigger for r in indicators.get_fisher_transform(quotes, 10)]
df['bullishFisher'] = 0.0
df['bullishFisher'] = np.where(df['fisher'] > df['fisher_trigger'], 1.0, 0.0)
df['crossover_Fisher'] = df['bullishFisher'].diff()

# Choppiness Index
df['chopp'] = [r.chop for r in indicators.get_chop(quotes, 14)]

# Price Momentum Oscillator (PMO)
df['pmo'] = [r.pmo for r in indicators.get_pmo(quotes, 35, 20, 10)]
df['pmo_signal'] = [r.signal for r in indicators.get_pmo(quotes, 35, 20, 10)]
df['bullishPMO'] = 0.0
df['bullishPMO'] = np.where(df['pmo'] > df['pmo_signal'], 1.0, 0.0)
df['crossover_PMO'] = df['bullishPMO'].diff()

# Calculate RSI
df['rsi'] = [r.rsi for r in indicators.get_rsi(quotes, 14)]
df['rsima6'] = df['rsi'].rolling(6).mean()
df['rsima14'] = df['rsi'].rolling(14).mean()
df['rsi_bullish'] = 0.0
df['rsi_bullish'] = np.where(df['rsi'] > df['rsima14'], 1.0, 0.0)
df['crossover_rsi'] = df['rsi_bullish'].diff()

#True Strength Index (TSI)
df['tsi'] = [r.tsi for r in indicators.get_tsi(quotes, 25, 13)]
df['tsi_signal'] = [r.signal for r in indicators.get_tsi(quotes, 25, 13)]
df['bullishTSI'] = 0.0
df['bullishTSI'] = np.where(df['tsi'] > df['tsi_signal'], 1.0, 0.0)
df['crossover_TSI'] = df['bullishTSI'].diff()

# Calculate Supertrend
df['supertrend'] = [r.super_trend for r in indicators.get_super_trend(quotes, 14, 3)]
df['supertrend_direction'] = 0.0
df['supertrend_direction'] = np.where(df['supertrend'] > df['Close'], 0.0, 1.0)
df['crossover_supertrend'] = df['supertrend_direction'].diff()

# Calculate Mcginley dynamic
df['dynamic20'] = [r.dynamic for r in indicators.get_dynamic(quotes, 20)]
df['bullishMcGinley'] = 0.0
df['bullishMcGinley'] = np.where(df['Close'] > df['dynamic20'], 1.0, 0.0)
df['crossover_McGinley'] = df['bullishMcGinley'].diff()


In [5]:
df['atr_stop'] = df['atr_stop'].astype(str) 
df['supertrend'] = df['supertrend'].astype(str)

# Now you can serialize your DataFrame
df.to_json() 


df = df.dropna().reset_index(drop=True)
df.head()


,Date,Close,High,Low,Open,Volume,atr_stop,bullishATRStop,crossover_ATRStop,adx,...,tsi,tsi_signal,bullishTSI,crossover_TSI,supertrend,supertrend_direction,crossover_supertrend,dynamic20,bullishMcGinley,crossover_McGinley
0,2010-04-06,89.944778,90.103450,89.378090,89.476314,110384200,87.1186281057373,1.0,0.0,28.113872,...,43.453603,41.120561,1.0,0.0,87.0394746466206,1.0,0.0,87.378881,1.0,0.0
1,2010-04-07,89.430984,90.186569,89.015411,89.763444,184576300,87.1186281057373,1.0,0.0,28.177650,...,42.620059,41.495435,1.0,0.0,87.0394746466206,1.0,0.0,87.534726,1.0,0.0
2,2010-04-08,89.740791,89.891912,88.856759,89.121212,158704000,87.1186281057373,1.0,0.0,27.959898,...,42.444534,41.732710,1.0,0.0,87.0394746466206,1.0,0.0,87.701144,1.0,0.0
3,2010-04-09,90.330124,90.367900,89.763436,89.929659,133006500,87.5066113302494,1.0,0.0,28.231094,...,43.232163,42.107573,1.0,0.0,87.3461132824918,1.0,0.0,87.895812,1.0,0.0
4,2010-04-12,90.473694,90.707929,90.337688,90.443470,110279000,87.7306621017593,1.0,0.0,28.793705,...,44.080747,42.600867,1.0,0.0,87.9165500078466,1.0,0.0,88.087179,1.0,0.0


In [6]:
# ATR Trailing Stop
if __name__ == '__main__':

    chart = Chart(title="ATR Trailing Stop", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    atr_stop_line = chart.create_line('atr_stop', color="#d62728", width=1, price_line=False, price_label=False)
    atr_stop_line.set(df[['Date', 'atr_stop']])
   
   # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        atrStop = df.iloc[i]['crossover_ATRStop']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if atrStop == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif atrStop == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)

chart.show(block = True)

In [8]:
# Average Directional Index (ADX)
if __name__ == '__main__':
    
    chart = Chart(title="ADX Crossover", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    adx_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    adx_line = adx_chart.create_line('adx', color="#e3b90f", width=1, price_line=False, price_label=False)
    adx_line.set(df[['Date', 'adx']])
    adx_pdi_line = adx_chart.create_line('adx_pdi', color="#75f321", width=1, price_line=False, price_label=False)
    adx_pdi_line.set(df[['Date', 'adx_pdi']])
    adx_mdi_line = adx_chart.create_line('adx_mdi', color="#f40707", width=1, price_line=False, price_label=False)
    adx_mdi_line.set(df[['Date', 'adx_mdi']])   

    


    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        adx_diff = df.iloc[i]['crossover_ADX']
        adx_value = df.iloc[i]['adx']
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if adx_diff == 1  and adx_value > 25:
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif adx_diff == -1 and adx_value > 25:
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)

In [ ]:
# Vortex Indicator (VI)
if __name__ == '__main__':
    
    chart = Chart(title="VI Crossover", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    vi_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    vi_pvi_line = vi_chart.create_line('pvi', color="#75f321", width=1, price_line=False, price_label=False)
    vi_pvi_line.set(df[['Date', 'pvi']])
    vi_mvi_line = vi_chart.create_line('mvi', color="#f40707", width=1, price_line=False, price_label=False)
    vi_mvi_line.set(df[['Date', 'mvi']])
    adx_mdi_line.set(df[['Date', 'adx_mdi']])   

    
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        vi_diff = df.iloc[i]['crossover_VI']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if vi_diff == 1:
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif vi_diff == -1:
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)

In [ ]:
# Moving Average Convergence / Divergence (MACD)
if __name__ == '__main__':
    
    chart = Chart(title="MACD Crossover", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    macd_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    macd_line = macd_chart.create_line('macd', color="#2ee30f", width=1, price_line=False, price_label=False)
    macd_line.set(df[['Date', 'macd']])

    signal_line = macd_chart.create_line('signal', color="#f36021", width=1, price_line=False, price_label=False)
    signal_line.set(df[['Date', 'signal']])

    histogram_bar = macd_chart.create_histogram('histogram', color="#1341e4", price_line=False, price_label=False)
    histogram_bar.set(df[['Date', 'histogram']])


    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        MACD_diff = df.iloc[i]['crossover_MACD']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if MACD_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif MACD_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)

In [ ]:
# Williams Alligator
if __name__ == '__main__':
    
    chart = Chart(title="Williams Alligator", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for Williams Alligator
    jaw_line = chart.create_line('jaw', color="#1f77b4", width=1, price_line=False, price_label=False)
    jaw_line.set(df[['Date', 'jaw']])
    teeth_line = chart.create_line('teeth', color="#ff7f0e", width=1, price_line=False, price_label=False)
    teeth_line.set(df[['Date', 'teeth']])
    lips_line = chart.create_line('lips', color="#2ca02c", width=1, price_line=False, price_label=False)
    lips_line.set(df[['Date', 'lips']])

chart.show(block = True)

In [ ]:
# Bollinger Bands
if __name__ == '__main__':

    chart = Chart(title="Bollinger Bands", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    upper_band_line = chart.create_line('BB_upper_band', color="#ff0000", width=1, price_line=False, price_label=False)
    upper_band_line.set(df[['Date', 'BB_upper_band']])
    middle_band_line = chart.create_line('BB_middle_band', color="#f3bd0b", width=1, price_line=False, price_label=False)
    middle_band_line.set(df[['Date', 'BB_middle_band']])
    lower_band_line = chart.create_line('BB_lower_band', color="#00ff00", width=1, price_line=False, price_label=False)
    lower_band_line.set(df[['Date', 'BB_lower_band']])

chart.show(block = True)

In [ ]:
# Stochastic Momentum Index (SMI)
if __name__ == '__main__':
    
    chart = Chart(title="Stochastic Momentum Index (SMI)", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    smi_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    smi_line = smi_chart.create_line('smi', color="#2ee30f", width=1, price_line=False, price_label=False)
    smi_line.set(df[['Date', 'smi']])

    signal_line = smi_chart.create_line('smi_signal', color="#f36021", width=1, price_line=False, price_label=False)
    signal_line.set(df[['Date', 'smi_signal']])

    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        SMI_diff = df.iloc[i]['crossover_SMI']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if SMI_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif SMI_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)

In [ ]:
# Chandelier Exit

if __name__ == '__main__':
    
    chart = Chart(title="chandelier long_exit and short_exit Crossover", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Add Chandelier Exit lines
    chandelier_long_line = chart.create_line('Chandelier_Long_Exit', color='#8e44ad', width=1, price_line=False, price_label=False)
    chandelier_long_line.set(df[['Date', 'chandelier_long_exit']])
    # chandelier_short_line = chart.create_line('Chandelier_Short_Exit', color='#e74c3c', width=1, price_line=False, price_label=False)
    # chandelier_short_line.set(df[['Date', 'chandelier_short_exit']])

    
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        chan_diff = df.iloc[i]['crossover_chandelier_exit']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if chan_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif chan_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)


In [ ]:
# Now, let's create the chart with lightweight-charts and add the EMAs and buy/sell markers.

# Assuming `df` is already a pandas DataFrame with 'Date', 'Open', 'High', 'Low', 'Close', 'EMA 12', and 'EMA 25' columns.
# It's good practice to convert the 'Date' column to the correct datetime format.
# df['Date'] = pd.to_datetime(df['Date'])

if __name__ == '__main__':
    
    chart = Chart(title="EMA12_EMA25 Crossover", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    ema12_line = chart.create_line('EMA 12', color='#ffeb3b', width=1, price_line=False, price_label=False)
    ema12_line.set(df[['Date', 'EMA 12']])

    ema25_line = chart.create_line('EMA 25', color='#26c6da', width=1, price_line=False, price_label=False)
    ema25_line.set(df[['Date', 'EMA 25']])
              
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        emadiff = df.iloc[i]['crossover_EMA12_EMA25']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if emadiff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif emadiff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)


In [ ]:
# Arnaud Legoux Moving Average (ALMA)

if __name__ == '__main__':
    
    chart = Chart(title="Arnaud Legoux Moving Average (ALMA)", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)
    # chart.layout(background_color="white")

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    alma21_line = chart.create_line('ALMA_21', color='#ffeb3b', width=1, price_line=False, price_label=False)
    alma21_line.set(df[['Date', 'ALMA_21']])

    alma50_line = chart.create_line('ALMA_50', color='#26c6da', width=1, price_line=False, price_label=False)
    alma50_line.set(df[['Date', 'ALMA_50']])
              
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        alma_diff = df.iloc[i]['crossover_ALMA']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if alma_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif alma_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)


In [ ]:
# Ehlers Fisher Transform

if __name__ == '__main__':
    
    chart = Chart(title="Ehlers Fisher Transform", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)
    # chart.layout(background_color="white")

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for fisher transform
    fisher_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    fisher_line = fisher_chart.create_line('fisher', color='#ffeb3b', width=1, price_line=False, price_label=False)
    fisher_line.set(df[['Date', 'fisher']])

    fisher_trigger_line = fisher_chart.create_line('fisher_trigger', color='#26c6da', width=1, price_line=False, price_label=False)
    fisher_trigger_line.set(df[['Date', 'fisher_trigger']])
              
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        fisher_diff = df.iloc[i]['crossover_Fisher']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if fisher_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif fisher_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)


In [ ]:
# Choppiness Index

if __name__ == '__main__':

    chart = Chart(title="Choppiness Index", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)
    # chart.layout(background_color="white")

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for choppiness index
    chop_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    chop_line = chop_chart.create_line('chopp', color='#ffeb3b', width=1, price_line=False, price_label=False)
    chop_line.set(df[['Date', 'chopp']])
    # chop60_line = chop_chart.horizontal_line(price=60, color='#26c6da', width=1, style='dashed', text='60')
    # chop40_line = chop_chart.horizontal_line(price=40, color='#ff5733', width=1, style='dashed', text='40')

chart.show(block = True)


In [ ]:
# Price Momentum Oscillator (PMO)

if __name__ == '__main__':

    chart = Chart(title="Price Momentum Oscillator (PMO)", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)
    # chart.layout(background_color="white")

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for PMO
    pmo_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    pmo_line = pmo_chart.create_line('pmo', color='#ffeb3b', width=1, price_line=False, price_label=False)
    pmo_line.set(df[['Date', 'pmo']])

    pmo_signal_line = pmo_chart.create_line('pmo_signal', color='#26c6da', width=1, price_line=False, price_label=False)
    pmo_signal_line.set(df[['Date', 'pmo_signal']])
              
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        pmo_diff = df.iloc[i]['crossover_PMO']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if pmo_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif pmo_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
chart.show(block = True)


In [ ]:
# True Strength Index (TSI)
if __name__ == '__main__':

    chart = Chart(title="True Strength Index (TSI)", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)
    # chart.layout(background_color="white")

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for TSI
    tsi_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    tsi_line = tsi_chart.create_line('tsi', color='#ffeb3b', width=1, price_line=False, price_label=False)
    tsi_line.set(df[['Date', 'tsi']])

    tsi_signal_line = tsi_chart.create_line('tsi_signal', color='#26c6da', width=1, price_line=False, price_label=False)
    tsi_signal_line.set(df[['Date', 'tsi_signal']])
              
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        tsi_diff = df.iloc[i]['crossover_TSI']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if tsi_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif tsi_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)

chart.show(block=True)

In [ ]:
df['supertrend'] = df['supertrend'].astype(str)

# Now you can serialize your DataFrame
df.to_json() 

In [ ]:
# Supertrend
if __name__ == '__main__':

    chart = Chart(title="Supertrend", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)
    # chart.layout(background_color="white")

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for Supertrend

    supertrend_line = chart.create_line('supertrend', color='#ffeb3b', width=1, price_line=False, price_label=False)
    supertrend_line.set(df[['Date', 'supertrend']])

    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        supertrend_diff = df.iloc[i]['crossover_supertrend']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if supertrend_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif supertrend_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)

chart.show(block=True)

In [ ]:
# Supertrend and McGinley Dynamic
if __name__ == '__main__':

    chart = Chart(title="Supertrend and McGinley Dynamic", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)
    # chart.layout(background_color="white")

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for Supertrend

    supertrend_line = chart.create_line('supertrend', color='#ffeb3b', width=1, price_line=False, price_label=False)
    supertrend_line.set(df[['Date', 'supertrend']])

    dynamic20_line = chart.create_line('dynamic20', color='#26c6da', width=1, price_line=False, price_label=False)
    dynamic20_line.set(df[['Date', 'dynamic20']])

    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        supertrend_diff = df.iloc[i]['crossover_supertrend']
        dynamic20_diff = df.iloc[i]['crossover_McGinley']
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if supertrend_diff == 1 and dynamic20_diff == 1:
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif supertrend_diff == -1 and dynamic20_diff == -1:
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)

chart.show(block=True)

In [ ]:
# Supertrend and McGinley Dynamic and RSI
if __name__ == '__main__':

    chart = Chart(title="Supertrend and McGinley Dynamic and RSI", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)
    # chart.layout(background_color="white")

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for Supertrend

    supertrend_line = chart.create_line('supertrend', color='#ffeb3b', width=1, price_line=False, price_label=False)
    supertrend_line.set(df[['Date', 'supertrend']])

    dynamic20_line = chart.create_line('dynamic20', color='#26c6da', width=1, price_line=False, price_label=False)
    dynamic20_line.set(df[['Date', 'dynamic20']])

    ris_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    rsi_line = ris_chart.create_line('rsi', color='#ff5733', width=1, price_line=False, price_label=False)
    rsi_line.set(df[['Date', 'rsi']])

    rsi_ma14_line = ris_chart.create_line('rsima14', color='#33ff57', width=1, price_line=False, price_label=False)
    rsi_ma14_line.set(df[['Date', 'rsima14']])

    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        supertrend_diff = df.iloc[i]['crossover_supertrend']
        dynamic20_diff = df.iloc[i]['crossover_McGinley']
        rsi_diff = df.iloc[i]['crossover_rsi']
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if supertrend_diff == 1 and dynamic20_diff == 1 and rsi_diff == 1:
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif supertrend_diff == -1 and dynamic20_diff == -1 and rsi_diff == -1:
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)

chart.show(block=True)

In [ ]:
# Supertrend and RSI
if __name__ == '__main__':

    chart = Chart(title="Supertrend and RSI", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)
    # chart.layout(background_color="white")

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for Supertrend

    supertrend_line = chart.create_line('supertrend', color='#ffeb3b', width=1, price_line=False, price_label=False)
    supertrend_line.set(df[['Date', 'supertrend']])

    
    ris_chart = chart.create_subchart(position='left', width=1.0, height=0.2, sync=True)
    rsi_line = ris_chart.create_line('rsi', color='#ff5733', width=1, price_line=False, price_label=False)
    rsi_line.set(df[['Date', 'rsi']])

    rsi_ma14_line = ris_chart.create_line('rsima14', color='#33ff57', width=1, price_line=False, price_label=False)
    rsi_ma14_line.set(df[['Date', 'rsima14']])

    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        supertrend_diff = df.iloc[i]['crossover_supertrend']
        rsi_diff = df.iloc[i]['crossover_rsi']
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if supertrend_diff == 1 and rsi_diff == 1:
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif supertrend_diff == -1 and rsi_diff == -1:
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)

chart.show(block=True)